In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%pylab inline

In [ ]:
import sys, argparse, logging
import pymssql
import _mssql
from datetime import datetime, time
from decimal import Decimal
import pymongo
from pymongo import MongoClient
from tqdm import tqdm, tnrange, tqdm_notebook
import pandas as pd
from tmqr.settings import *
import pickle
try:
    from tmqr.settings_local import *
except:
    pass

from tmqrfeed.datafeed import DataFeed



In [ ]:
# Init mongo asset index
client = MongoClient(MONGO_CONNSTR)
mongo_db = client[MONGO_DB]


In [ ]:
feed = DataFeed()

In [ ]:
chain = feed.get_fut_chain('US.CL')

In [ ]:
futures = chain.get_all()

In [ ]:
fut = futures[0]

In [ ]:
print('Contract: {0} SqlID: {1}'.format(fut, fut.contract_info.extra('sqlid')))

In [ ]:
# Storing futures
mongo_collection = mongo_db['tmp_bardata']
mongo_collection.create_index([('idcontract', pymongo.ASCENDING), ('datetime', pymongo.ASCENDING)])

In [ ]:
quotes_collection = mongo_db['quotes_intraday']
quotes_collection.create_index([('tckr', pymongo.ASCENDING), ('dt',pymongo.ASCENDING)], unique=True)

In [ ]:
for fut in tqdm_notebook(futures):
    data = list(mongo_collection.find({'idcontract': fut.contract_info.extra('sqlid')}))
    if len(data) == 0:
        print("Empty contract series for {0} ... skipping".format(fut))
        continue
    df = pd.DataFrame(data)
    df = df[['datetime', 'open', 'high', 'low', 'close', 'volume']]
    df.rename(columns={'datetime': 'dt', 'open': 'o', 'high': 'h', 'low':'l', 'close': 'c', 'volume': 'v'}, inplace=True)
    df.set_index('dt', inplace=True)
    df.index = df.index.tz_localize(fut.instrument_info.timezone).tz_convert('UTC')
    
    for idx_dt, df_value in df.groupby(by=df.index.date):
        dt = datetime.combine(idx_dt, time(0,0,0))
        rec = {
            'dt': dt,
            'tckr': fut.ticker,
            'ohlc': pickle.dumps(df_value)
            }
        quotes_collection.replace_one({'dt': dt, 'tckr': fut.ticker}, rec, upsert=True)    
